This notebook is based on a [blog](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) Francois Chollet has written mid 2016, please refer to that one of more information. In addition, I recommend reading [this](http://adventuresinmachinelearning.com/word2vec-keras-tutorial/) tutorial before to get started on word2vec



First we'll download and exract the necessary files; the glove embeddings on the wikipedia corpus from [Stanford](https://nlp.stanford.edu/projects/glove/) as well as [newsgroup messages](http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html) used for classification

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!wget http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz
!tar xvfz news20.tar.gz

source taken from https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py

In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

/gpfs/fs01/user/se2e-1a118c4f322670-980ba6aaa6c3/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

first, build index mapping words in the embeddings set to their embedding vector

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


prepare text samples and their labels

In [6]:
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir('20_newsgroup')):
    path = os.path.join('20_newsgroup', name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Processing text dataset
Found 19997 texts.


vectorize the text samples into a 2D integer tensor

In [11]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 174105 unique tokens.


In [12]:
print(len(sequences))
print(len(sequences[0]))
print(len(sequences[1]))

19997
1528
5116


In [13]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


split the data into a training set and a validation set , first, shuffle data

In [17]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

then split based on split percentage stored in VALIDATION_SPLIT

In [18]:
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [11]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

#Untrained Embedding:
#embedding_layer = Embedding(len(word_index) + 1,
#                            EMBEDDING_DIM,
#                            input_length=MAX_SEQUENCE_LENGTH)

Preparing embedding matrix.


In [41]:
embeddings_index.get('the')

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [12]:


print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(embedded_sequences)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=30,
          validation_data=(x_val, y_val))

print("Training done")


Training model.
Train on 15998 samples, validate on 3999 samples
Epoch 1/30
15998/15998 [==============================] - 81s 5ms/step - loss: 2.9310 - acc: 0.1970 - val_loss: 2.7595 - val_acc: 0.3701
Epoch 2/30
15998/15998 [==============================] - 82s 5ms/step - loss: 2.2948 - acc: 0.4995 - val_loss: 1.8832 - val_acc: 0.5699
Epoch 3/30
15998/15998 [==============================] - 81s 5ms/step - loss: 1.4982 - acc: 0.6572 - val_loss: 1.3274 - val_acc: 0.6569
Epoch 4/30
15998/15998 [==============================] - 80s 5ms/step - loss: 1.0370 - acc: 0.7355 - val_loss: 1.0437 - val_acc: 0.7034
Epoch 5/30
15998/15998 [==============================] - 81s 5ms/step - loss: 0.7835 - acc: 0.7878 - val_loss: 0.8983 - val_acc: 0.7344
Epoch 6/30
15998/15998 [==============================] - 76s 5ms/step - loss: 0.6242 - acc: 0.8295 - val_loss: 0.8134 - val_acc: 0.7539
Epoch 7/30
15998/15998 [==============================] - 72s 4ms/step - loss: 0.5095 - acc: 0.8579 - val_loss: 0

In [13]:
preds = model.predict(x_val)

In [14]:
np.sum(np.argmax(preds,axis=1) == np.argmax(y_val,axis=1)) / float(len(preds))


0.7974493623405852

In [ ]:
x_train

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [19]:
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

In [20]:
model = loadGloveModel('glove.6B.100d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [22]:
concept_of_queen = model['king']-model['man']+model['woman']
np.sum((concept_of_queen-model['queen'])**2)

16.65520237287384

In [25]:
concept_of_problems = model['iraq']-model['war']
np.sum((concept_of_problems-model['iran'])**2)

45.17898720772049